In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime

import tensorflow as tf
from tensorflow import keras

In [3]:
import tensorboard
tensorboard.__version__

'2.2.1'

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/ 


In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])


In [6]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0


In [7]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])


Epoch 1/5
938/938 [==============================] - 18s 20ms/step - loss: 0.6741 - accuracy: 0.7678
Epoch 2/5
938/938 [==============================] - 19s 20ms/step - loss: 0.4832 - accuracy: 0.8296
Epoch 3/5
938/938 [==============================] - 22s 23ms/step - loss: 0.4459 - accuracy: 0.8393
Epoch 4/5
938/938 [==============================] - 17s 19ms/step - loss: 0.4252 - accuracy: 0.8465
Epoch 5/5
938/938 [==============================] - 12s 13ms/step - loss: 0.4103 - accuracy: 0.8514


In [9]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6011 (pid 6390), started 0:03:32 ago. (Use '!kill 6390' to kill it.)

In [10]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)


In [11]:
%tensorboard --logdir logs/func